In [1]:
import pandas as pd
import wrds
import os
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from scipy.stats import ttest_1samp
from scipy.stats import kurtosis, skew
import pandas as pd
from scipy.stats import skew, kurtosis
from scipy.stats import mstats

In [2]:
db = wrds.Connection(wrds_username='charly99')

Loading library list...
Done


In [3]:
# Define the input directory
input_directory = '/work/pi_atreya_chakraborty_umb_edu/Captsone/Data'

# Define the input file paths
clean_analyst_path_weekly = os.path.join(input_directory, 'clean_analyst_weekly.dta')
clean_analyst_path_monthly = os.path.join(input_directory, 'clean_analyst_monthly.dta')
clean_crsp_path = os.path.join(input_directory, 'clean_crsp.dta')

# Read the datasets
results_df_weekly = pd.read_stata(clean_analyst_path_weekly)
results_df_monthly = pd.read_stata(clean_analyst_path_monthly)
crsp_data = pd.read_stata(clean_crsp_path)

results_df_weekly = results_df_weekly.groupby(['cusip', 'estimid', 'year', 'month', 'week']).first().reset_index()
results_df_monthly = results_df_monthly.groupby(['cusip', 'estimid', 'year', 'month']).first().reset_index()

# Convert 'ireccd' to numeric, forcing errors to NaN (if any)
results_df_monthly['ireccd'] = pd.to_numeric(results_df_monthly['ireccd'], errors='coerce')

# Convert 'ireccd' to numeric, forcing errors to NaN (if any)
results_df_weekly['ireccd'] = pd.to_numeric(results_df_monthly['ireccd'], errors='coerce')

In [4]:
# Perform the transformation (6 - ireccd)
results_df_monthly['ireccd_transformed'] = 6 - results_df_monthly['ireccd']

# Define a function to calculate descriptive statistics, including Q1, Q3, kurtosis, and skewness
def get_descriptive_stats(group):
    return pd.Series({
        'mean': group.mean(),
        'median': group.median(),
        'std': group.std(),
        'min': group.min(),
        'max': group.max(),
        'count': group.count(),
        'skewness': skew(group, nan_policy='omit'),
        'kurtosis': kurtosis(group, nan_policy='omit'),
        'Q1': group.quantile(0.25),  # First quartile (25th percentile)
        'Q3': group.quantile(0.75)   # Third quartile (75th percentile)
    })

# Apply the descriptive statistics function to each industry in ff_17
descriptive_stats_by_ff_17 = results_df_monthly.groupby('ff_17')['ireccd_transformed'].apply(get_descriptive_stats).unstack()

# Calculate the overall descriptive statistics across all industries
overall_stats = get_descriptive_stats(results_df_monthly['ireccd_transformed'])
overall_stats_df = pd.DataFrame(overall_stats).T
overall_stats_df.index = ['Overall']

# Append the overall statistics at the bottom of the industry-specific statistics
final_stats_df = pd.concat([descriptive_stats_by_ff_17, overall_stats_df])

# Save the final dataframe with descriptive statistics to a CSV file
final_stats_df.to_csv("Industry_Descriptive_with_Quartiles.csv")

In [5]:
final_stats_df.head()

,mean,median,std,min,max,count,skewness,kurtosis,Q1,Q3
Cars,3.620951,4.0,0.958353,1.0,5.0,52447.0,-0.143646,-0.350782,3.0,4.0
Chems,3.685024,4.0,0.960687,1.0,5.0,183871.0,-0.293608,-0.188068,3.0,4.0
Clths,3.701181,4.0,0.933321,1.0,5.0,31427.0,-0.169168,-0.366749,3.0,4.0
Cnstr,3.676580,4.0,0.951167,1.0,5.0,87119.0,-0.192823,-0.353264,3.0,4.0
Cnsum,3.740731,4.0,0.936590,1.0,5.0,11949.0,-0.202912,-0.485330,3.0,5.0


In [6]:
# Now, recode 'ireccd' so that higher values imply a more favorable recommendation
results_df_monthly['ireccd'] = 6 - results_df_monthly['ireccd']

# After recoding, proceed with the calculation of the average 'ireccd' as described earlier
monthly_rec = results_df_monthly.groupby(['cusip', 'year', 'month']).agg(
    avg_ireccd=('ireccd', 'mean'),
    sic=('sic', 'first'),
    ff_5=('ff_5', 'first'),
    ff_10=('ff_10', 'first'),
    ff_17=('ff_17', 'first'),
    ff_48=('ff_48', 'first')
).reset_index()

In [7]:
# Sort crsp_data by 'cusip' and 'dlycaldt'
crsp_data = crsp_data.sort_values(by=['cusip', 'dlycaldt'])

# Create a 'year_month' column to group by year and month
crsp_data['year_month'] = crsp_data['dlycaldt'].dt.to_period('M')

# Calculate the average daily capitalization for each month
crsp_data['monthly_avg_cap'] = crsp_data.groupby(['cusip', 'year_month'])['dlycap'].transform('mean')

# Shift the monthly average capitalization by one period (one month) within each 'cusip'
crsp_data['prior_month_avg_cap'] = crsp_data.groupby('cusip')['monthly_avg_cap'].shift()

# Step 1: Calculate daily returns (percentage change in price)
crsp_data['daily_return'] = crsp_data.groupby('cusip')['dlyprc'].pct_change(fill_method=None)

# Step 2: Calculate absolute daily returns
crsp_data['abs_daily_return'] = np.abs(crsp_data['daily_return'])

# Step 3: Ensure that 'dlycap' (dollar volume) is non-zero and non-missing
crsp_data['dlycap'] = crsp_data['dlycap'].replace(0, np.nan)

# Step 4: Calculate the ILLIQ measure for each day as |return| / volume
crsp_data['illiq_daily'] = crsp_data['abs_daily_return'] / crsp_data['dlycap']

# Step 5: Aggregate ILLIQ to the monthly level
# Group by 'cusip' and 'year_month' to calculate the monthly ILLIQ as the mean of the daily values
illiq_df = crsp_data.groupby(['cusip', 'year_month']).agg(ILLIQ=('illiq_daily', 'mean')).reset_index()

# Step 6: Merge the calculated ILLIQ values back into the main dataset
crsp_data = crsp_data.merge(illiq_df, on=['cusip', 'year_month'], how='left')

# Step 7: Aggregate data to get the beginning and ending prices, prior month's average capitalization, and standard deviation
crsp_monthly = crsp_data.groupby(['cusip', 'year', 'month']).agg(
    beginning_price=('dlyprc', 'first'),
    ending_price=('dlyprc', 'last'),
    cusip9=('cusip9', 'first'),
    prior_month_avg_cap=('prior_month_avg_cap', 'first'),
    IVOL=('dlyprc', 'std'),
    ILLIQ=('ILLIQ', 'first')
).reset_index()

# Step 8: Calculate the monthly return
crsp_monthly['month_return'] = (crsp_monthly['ending_price'] - crsp_monthly['beginning_price']) / crsp_monthly['beginning_price'] * 100

# Winsorize the 'month_return' at the 1% and 99% levels
crsp_monthly['month_return_winsorized'] = mstats.winsorize(crsp_monthly['month_return'], limits=[0.01, 0.01])

# Shift the monthly return by one period (one month) within each 'cusip'
crsp_monthly['Rt_1'] = crsp_monthly.groupby('cusip')['month_return_winsorized'].shift(1)

# Calculate Rt−12,t−2 (Cumulative return from t−12 to t−2)
def cumulative_return(group, start_shift, end_shift):
    return group.shift(end_shift).rolling(window=(start_shift - end_shift)).sum()

crsp_monthly['Rt_12_t_2'] = crsp_monthly.groupby('cusip')['month_return_winsorized'].transform(
    lambda x: cumulative_return(x, 12, 2))

# Calculate Rt−60,t−13 (Cumulative return from t−60 to t−13)
crsp_monthly['Rt_60_t_13'] = crsp_monthly.groupby('cusip')['month_return_winsorized'].transform(
    lambda x: cumulative_return(x, 60, 13))

# Shift the monthly illiquidity measure to the prior month by one period (one month) within each 'cusip'
crsp_monthly['ILLIQ_prior'] = crsp_monthly.groupby('cusip')['ILLIQ'].shift()

# Shift the monthly volatility measure to the prior month by one period (one month) within each 'cusip'
crsp_monthly['IVOL_prior'] = crsp_monthly.groupby('cusip')['IVOL'].shift()

# Define the function to assign periods based on the year
def assign_period(year):
    if 1992 <= year <= 1999:
        return 1
    elif 2000 <= year <= 2009:
        return 2
    elif 2010 <= year <= 2019:
        return 3
    elif 2020 <= year <= 2024:
        return 4
    else:
        return None

# Apply the function to create the 'period' column
crsp_monthly['period'] = crsp_monthly['year'].apply(assign_period)

In [16]:
# Merge monthly_rec with crsp_monthly based on 'cusip', 'year', and 'month'
rec_return = pd.merge(monthly_rec, crsp_monthly[['cusip', 'cusip9', 'year', 'month', 'month_return', 'month_return_winsorized', 'IVOL', 'IVOL_prior', 'prior_month_avg_cap', 'ILLIQ', 'ILLIQ_prior', 'Rt_1', 'Rt_12_t_2', 'Rt_60_t_13']], 
                    on=['cusip', 'year', 'month'], 
                    how='left')

In [17]:
# File paths
ff_momentum_path = f"{input_directory}/FF_Momentum_Monthly.csv"
ff_str_path = f"{input_directory}/FF_STR_Monthly.csv"
ff3_path = f"{input_directory}/FF3_Monthly.csv"
ff5_path = f"{input_directory}/FF5_Monthly.csv"
q_factor_path = f"{input_directory}/q_factor.csv"

# Function to split Year-Month into separate year and month columns
def add_year_month(df, col_name='Year-Month'):
    df['year'] = df[col_name].astype(str).str[:4].astype(int)
    df['month'] = df[col_name].astype(str).str[4:].astype(int)
    return df

# Load the datasets
ff_momentum = pd.read_csv(ff_momentum_path)
ff_str = pd.read_csv(ff_str_path)
ff3 = pd.read_csv(ff3_path)
ff5 = pd.read_csv(ff5_path)
q_factor = pd.read_csv(q_factor_path)

# Add year and month columns to each file
ff_momentum = add_year_month(ff_momentum)
ff_str = add_year_month(ff_str)
ff3 = add_year_month(ff3)
ff5 = add_year_month(ff5)

# Ensure rec_return dataframe has year and month columns
if 'year' not in rec_return.columns or 'month' not in rec_return.columns:
    rec_return['year'] = rec_return['date'].astype(str).str[:4].astype(int)
    rec_return['month'] = rec_return['date'].astype(str).str[5:7].astype(int)

# Merge the four datasets into rec_return using lowercase year and month
rec_return = pd.merge(rec_return, ff_momentum, on=['year', 'month'], how='left')
rec_return = pd.merge(rec_return, ff_str, on=['year', 'month'], how='left')

# Merge FF3_Monthly with suffix for overlapping columns
rec_return = pd.merge(rec_return, ff3, on=['year', 'month'], how='left', suffixes=('', '_ff3'))

# Merge FF5_Monthly with suffix for overlapping columns
rec_return = pd.merge(rec_return, ff5, on=['year', 'month'], how='left', suffixes=('', '_ff5'))

# Merge q_factor_monthly with suffix for overlapping columns
rec_return = pd.merge(rec_return, q_factor, on=['year', 'month'], how='left', suffixes=('', '_q'))

In [19]:
# Ensure the winsorized columns are converted to numeric data type
rec_return['month_return_winsorized'] = pd.to_numeric(rec_return['month_return_winsorized'], errors='coerce')

rec_return.to_stata("rec_return.dta")

/tmp/ipykernel_355139/1370092072.py:4: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Year-Month_x   ->   Year_Month_x
    Mom      ->   Mom___
    Year-Month_y   ->   Year_Month_y
    Year-Month   ->   Year_Month
    Mkt-RF   ->   Mkt_RF
    Year-Month_ff5   ->   Year_Month_ff5
    Mkt-RF_ff5   ->   Mkt_RF_ff5

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  rec_return.to_stata("rec_return.dta")


In [20]:
# Download data from WRDS
yearly_data = db.raw_sql('''
    SELECT gvkey, cusip, datadate, fyear, at, mkvalt, sstk, dv, dvt, ceq 
    FROM comp.funda
    WHERE fyear >= 1990 AND indfmt = 'INDL' AND datafmt = 'STD' AND popsrc = 'D' AND consol = 'C' AND datadate <= '2024-06-01'
''', date_cols=['datadate'])

# Download quarterly data - No advertising for quarterly
quarterly_data = db.raw_sql('''
    SELECT gvkey, cusip, datadate, fqtr, fyearq, atq, revtq, oancfy, cogsq, invtq, xrdq, xsgaq, apalchy, mkvaltq, ceqq, actq, cheq, lctq, dlcq, txpq, dpq, dlttq, ppentq, mibq, pstkq, oiadpq, ltq, ajexq, rectq, epspxq
    FROM comp.fundq
    WHERE datadate >= '1990-01-01' AND indfmt = 'INDL' AND datafmt = 'STD' AND popsrc = 'D' AND consol = 'C' AND datadate <= '2024-06-01'
''', date_cols=['datadate'])

#create filling date to prevent look-ahead bias
#see https://www.investor.gov/introduction-investing/investing-basics/glossary/form-10-k
def calculate_filing_date_year(row):
    market_cap = row['mkvalt']  # In millions    
    if market_cap >= 700:
        return row['datadate'] + pd.DateOffset(days=60)  # Large Accelerated Filer
    elif 75 <= market_cap < 700:
        return row['datadate'] + pd.DateOffset(days=75)  # Accelerated Filer
    else:
        return row['datadate'] + pd.DateOffset(days=90)  # Non-accelerated Filer
    
# Apply the function to create the 'filling_date' column
yearly_data['filling_date'] = yearly_data.apply(calculate_filing_date_year, axis=1)

def calculate_filing_date_qtr(row):
    market_cap = row['mkvaltq']  # In millions
    fqtr = row['fqtr']
    
    # Determine if it's a 10-K or 10-Q based on fqtr
    if fqtr == 4:  # 10-K
        if market_cap >= 700:
            return row['datadate'] + pd.DateOffset(days=60)  # Large Accelerated Filer
        elif 75 <= market_cap < 700:
            return row['datadate'] + pd.DateOffset(days=75)  # Accelerated Filer
        else:
            return row['datadate'] + pd.DateOffset(days=90)  # Non-accelerated Filer
    else:  # 10-Q (fqtr 1, 2, or 3)
        if market_cap >= 75:
            return row['datadate'] + pd.DateOffset(days=40)  # Large and Accelerated Filers
        else:
            return row['datadate'] + pd.DateOffset(days=45)  # Non-accelerated Filer

# Apply the function to create the 'filling_date' column
quarterly_data['filling_date'] = quarterly_data.apply(calculate_filing_date_qtr, axis=1)

In [21]:
# Reset the index and drop the current index to avoid inserting it as a column
yearly_data = yearly_data.reset_index(drop=True)
quarterly_data = quarterly_data.reset_index(drop=True)

# Calculate financial metrics for yearly data
yearly_data['BM_yearly'] = np.where(yearly_data['mkvalt'] > 0, np.log(yearly_data['ceq'] / yearly_data['mkvalt']), np.nan)
yearly_data['Size_yearly'] = np.where(yearly_data['mkvalt'] > 0, np.log(yearly_data['mkvalt']), np.nan)
yearly_data['AG'] = (yearly_data['at'] - yearly_data.groupby('gvkey')['at'].shift(1)) / yearly_data.groupby('gvkey')['at'].shift(1)
yearly_data['XFIN'] = (yearly_data['sstk'] - yearly_data['dv']) / yearly_data['at']

# Calculate financial metrics for quarterly data
quarterly_data['BM_qtr'] = np.where(quarterly_data['mkvaltq'] > 0, np.log(quarterly_data['ceqq'] / quarterly_data['mkvaltq']), np.nan)
quarterly_data['Size_qtr'] = np.where(quarterly_data['mkvaltq'] > 0, np.log(quarterly_data['mkvaltq']), np.nan)

quarterly_data['revtq_lag'] = quarterly_data.groupby('gvkey')['revtq'].shift(1)
quarterly_data['atq_lag'] = quarterly_data.groupby('gvkey')['atq'].shift(1)
quarterly_data['ltq_lag'] = quarterly_data.groupby('gvkey')['ltq'].shift(1)

quarterly_data['GrossProfit'] = (quarterly_data['revtq'] - quarterly_data['cogsq']) / quarterly_data.groupby('gvkey')['atq'].shift(1)

# CBOP Calculation
quarterly_data['delta_RECTQ'] = quarterly_data.groupby('gvkey')['rectq'].diff()
quarterly_data['delta_INVTQ'] = quarterly_data.groupby('gvkey')['invtq'].diff()

# CBOP calculation with APALCHY as the level
quarterly_data['CBOP'] = ((quarterly_data['revtq'] - quarterly_data['cogsq'] - quarterly_data['xsgaq'] + quarterly_data['xrdq']) - 
                          (quarterly_data['delta_RECTQ'] + quarterly_data['delta_INVTQ']) - 
                          quarterly_data['apalchy']) / quarterly_data.groupby('gvkey')['atq'].shift(1)


# Accruals Calculation
quarterly_data['delta_ACTQ'] = quarterly_data.groupby('gvkey')['actq'].diff()
quarterly_data['delta_CHEQ'] = quarterly_data.groupby('gvkey')['cheq'].diff()
quarterly_data['delta_LCTQ'] = quarterly_data.groupby('gvkey')['lctq'].diff()
quarterly_data['delta_DLCQ'] = quarterly_data.groupby('gvkey')['dlcq'].diff()
quarterly_data['delta_TXPQ'] = quarterly_data.groupby('gvkey')['txpq'].diff()

quarterly_data['Accruals'] = (quarterly_data['delta_ACTQ'] - quarterly_data['delta_CHEQ'] - 
                              (quarterly_data['delta_LCTQ'] - quarterly_data['delta_DLCQ'] - quarterly_data['delta_TXPQ']) - 
                              quarterly_data['dpq']) / quarterly_data.groupby('gvkey')['atq'].shift(1)

quarterly_data['WorkingCapital'] = (quarterly_data['actq'] - quarterly_data['lctq']) / quarterly_data['atq']
quarterly_data['STDebt'] = quarterly_data['dlcq'] / quarterly_data['atq']
quarterly_data['LTDebt'] = quarterly_data['dlttq'] / quarterly_data['atq']

quarterly_data['OpLev'] = (quarterly_data['cogsq'] + quarterly_data['xsgaq']) / quarterly_data['atq']
quarterly_data['CashHolding'] = quarterly_data['cheq'] / quarterly_data['atq']

quarterly_data['ZScore'] = (3.3 * quarterly_data['oiadpq'] + quarterly_data['revtq'] + 
                            1.4 * quarterly_data['ceqq'] + 
                            1.2 * (quarterly_data['actq'] - quarterly_data['lctq'])) / quarterly_data['atq']

quarterly_data['PPE'] = quarterly_data['ppentq'] / quarterly_data['atq']

# Calculate SUE
# Calculate the change in split-adjusted earnings per share (EPSPXQ divided by AJEXQ)
quarterly_data['eps_change'] = quarterly_data['epspxq'] / quarterly_data['ajexq']

# Calculate the earnings surprise (SUE)
quarterly_data['eps_lag4'] = quarterly_data.groupby('gvkey')['eps_change'].shift(4)

#Apply the rolling standard deviation for SUE calculation within each group
def calc_sue(group):
    # Calculate the rolling standard deviation of earnings per share
    group['eps_std'] = group['eps_change'].rolling(window=8, min_periods=6).std()
    # Calculate SUE
    group['SUE'] = (group['eps_change'] - group['eps_lag4']) / group['eps_std']
    return group

# Apply the function to each group (gvkey)
quarterly_data = quarterly_data.groupby('gvkey').apply(calc_sue)

# Calculate NOA
quarterly_data['NOA'] = ((quarterly_data['atq'] - quarterly_data['cheq']) - 
                         (quarterly_data['atq'] - quarterly_data['dlcq'] - quarterly_data['dlttq'] - 
                          quarterly_data['mibq'] - quarterly_data['pstkq'] - quarterly_data['ceqq'])) / quarterly_data['atq']

# Calculate R&D as a percentage of total assets
quarterly_data['R&D'] = quarterly_data['xrdq'] / quarterly_data['atq']

# Ensure 'gvkey' is not an index to avoid ambiguity issues, and avoid duplicate columns
if 'gvkey' in quarterly_data.index.names:
    quarterly_data = quarterly_data.reset_index(drop=True)

# Sales growth
quarterly_data['SalesGrowth'] = (quarterly_data['revtq'] - quarterly_data['revtq_lag'] ) /quarterly_data['revtq_lag'] 

# Return on Equity
quarterly_data['ROE'] = quarterly_data['oiadpq'] / (quarterly_data['/']  - quarterly_data['ltq_lag'])

# Asset turnover
quarterly_data['AssetTurnover'] = quarterly_data['revtq'] / quarterly_data['atq_lag'] 

# Profit margin
quarterly_data['ProfitMargin'] = quarterly_data['oiadpq'] / quarterly_data['revtq']

# Total leverage
quarterly_data['TotalLev'] = quarterly_data['ltq'] / quarterly_data['atq']

/home/uchenna_onuoha001_umb_edu/.local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/uchenna_onuoha001_umb_edu/.local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/uchenna_onuoha001_umb_edu/.local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/uchenna_onuoha001_umb_edu/.local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/uchenna_onuoha001_umb_edu/.local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwarg

KeyError: '/'

In [27]:
# Ensure 'year_month' column exists in rec_return as a combination of 'year' and 'month'
rec_return['year_month'] = pd.to_datetime(rec_return['year'].astype(str) + '-' + rec_return['month'].astype(str) + '-01')

# Identify duplicates based on 'cusip9' and 'year_month'
duplicates_df = rec_return[rec_return.duplicated(subset=['cusip9', 'year_month'], keep=False)]

# Display the duplicate rows
duplicates_df.head()


,cusip,year,month,avg_ireccd,sic,ff_5,ff_10,ff_17,ff_48,cusip9,...,RMW,CMA,RF_ff5,R_F,R_MKT,R_ME,R_IA,R_ROE,R_EG,year_month
0,00003605,2005,12,2.0,NaN,Other,Other,Other,Other,NaN,...,0.22,0.23,0.32,0.3160,-0.2427,-0.0336,0.8804,0.2287,1.0930,2005-12-01
1,00003605,2006,1,2.0,NaN,Other,Other,Other,Other,NaN,...,-0.65,-0.45,0.35,0.3488,3.0346,6.4413,-0.6797,-0.2611,0.8783,2006-01-01
2,00003605,2006,2,2.0,NaN,Other,Other,Other,Other,NaN,...,-0.51,1.91,0.34,0.3322,-0.2924,-0.3950,2.4381,-0.9978,-1.0257,2006-02-01
3,00003605,2006,3,2.0,NaN,Other,Other,Other,Other,NaN,...,0.06,-0.40,0.37,0.3642,1.4637,3.1221,-1.3435,0.7652,-0.7877,2006-03-01
42,00030710,2019,11,3.0,6799.0,Other,Other,Finan,Meals,NaN,...,-1.63,-1.25,0.12,0.1197,3.8673,0.0017,-1.0307,-1.3054,-0.6352,2019-11-01


In [26]:
# Ensure 'filling_date' exists and is of datetime type for both yearly and quarterly data
yearly_data['filling_date'] = pd.to_datetime(yearly_data['filling_date'])
quarterly_data['filling_date'] = pd.to_datetime(quarterly_data['filling_date'])

# Convert 'year' and 'month' columns in rec_return to datetime for proper comparison
rec_return['year_month'] = pd.to_datetime(rec_return['year'].astype(str) + '-' + rec_return['month'].astype(str) + '-01')

# Sort data by cusip and filling_date for proper asof merge behavior
yearly_data = yearly_data.sort_values(by=['cusip', 'filling_date'])
quarterly_data = quarterly_data.sort_values(by=['cusip', 'filling_date'])
rec_return = rec_return.sort_values(by=['cusip9', 'year_month'])

# Ensure both 'year_month' and 'filling_date' columns are sorted for asof merge
rec_return = rec_return.sort_values('year_month')
yearly_data = yearly_data.sort_values('filling_date')
quarterly_data = quarterly_data.sort_values('filling_date')

# Perform asof merge with yearly_data based on 'cusip9' from rec_return and 'cusip' from yearly_data
merged_df = pd.merge_asof(
    rec_return,
    yearly_data[['cusip', 'filling_date'] + yearly_data.columns.difference(['cusip', 'filling_date']).tolist()],
    left_on='year_month',        # Key for time comparison from rec_return
    right_on='filling_date',     # Key for time comparison from yearly_data
    left_by='cusip9',            # 'cusip9' from rec_return
    right_by='cusip',            # 'cusip' from yearly_data
    direction='backward'         # Ensure we get the last available data before the 'year_month'
)

# Perform asof merge with quarterly_data based on 'cusip9' from rec_return and 'cusip' from quarterly_data
merged_df = pd.merge_asof(
    merged_df,
    quarterly_data[['cusip', 'filling_date'] + quarterly_data.columns.difference(['cusip', 'filling_date']).tolist()],
    left_on='year_month',        # Key for time comparison from rec_return
    right_on='filling_date',     # Key for time comparison from quarterly_data
    left_by='cusip9',            # 'cusip9' from rec_return
    right_by='cusip',            # 'cusip' from quarterly_data
    direction='backward'         # Ensure we get the last available data before the 'year_month'
)

# Drop unnecessary 'filling_date' columns after merge
merged_df = merged_df.drop(columns=['filling_date_x', 'filling_date_y'], errors='ignore')

,cusip_x,year,month,avg_ireccd,sic,ff_5,ff_10,ff_17,ff_48,cusip9,...,oancfy,oiadpq,ppentq,pstkq,rectq,revtq,revtq_lag,txpq,xrdq,xsgaq
0,41135230,1992,12,5.0,6733.0,Other,Other,Finan,Meals,411352305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,41135230,1993,1,5.0,6733.0,Other,Other,Finan,Meals,411352305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41135230,1993,2,5.0,6733.0,Other,Other,Finan,Meals,411352305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,41135230,1993,3,5.0,6733.0,Other,Other,Finan,Meals,411352305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55942410,1993,10,5.0,3612.0,Manuf,Manuf,Machn,ElcEq,559424106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# Replace infinite values with NaN
merged_df = merged_df.replace([np.inf, -np.inf], np.nan)

# Now save to Stata file
merged_df.to_stata("rec_return_fama.dta")


/tmp/ipykernel_355139/2940942919.py:5: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Year-Month_x   ->   Year_Month_x
    Mom      ->   Mom___
    Year-Month_y   ->   Year_Month_y
    Year-Month   ->   Year_Month
    Mkt-RF   ->   Mkt_RF
    Year-Month_ff5   ->   Year_Month_ff5
    Mkt-RF_ff5   ->   Mkt_RF_ff5
    R&D   ->   R_D

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  merged_df.to_stata("rec_return_fama.dta")
